In [6]:
import torch
from darknet53_model import Darknet53  # 반드시 동일한 모델 정의 필요
import os
print(os.getcwd())
print(os.listdir("../model"))
# 설정
num_classes = 3
model_path = "../model/darknet53_gray.pth"

# 모델 정의 및 로드
model = Darknet53(num_classes=3)
model.load_state_dict(torch.load(model_path, map_location="cuda"))
model.eval()

print("✅ 모델이 성공적으로 로드되었습니다.")

model.eval()

c:\Users\user\Desktop\tasks\python\model_test\darknet_classification
['ckpt_latest.pth', 'darknet53_final.pth', 'darknet53_gray.pth']


C:\Users\user\AppData\Local\Temp\ipykernel_8164\3027697151.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location="cu

✅ 모델이 성공적으로 로드되었습니다.


Darknet53(
  (stem): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1, inplace=True)
  )
  (stage1): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1, inplace=True)
    (3): ResidualBlock(
      (block): Sequential(
        (0): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.1, inplace=True)
        (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): LeakyReLU

In [7]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model.to(device)
model.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])


image_dir = "../data/cropped"
output_dir = "../data/cropped_pred"
os.makedirs(output_dir, exist_ok=True)

# 이미지 파일만 필터링 (jpg, jpeg, png 등)
image_paths = [
    os.path.join(image_dir, fname)
    for fname in os.listdir(image_dir)
    if fname.lower().endswith(('.jpg', '.jpeg', '.png'))
]

class_names = ["smoke", "fire", "normal"]  # 실제 학습 시 클래스 순서대로 맞춰야 함


for img_path in image_paths:
    image = Image.open(img_path).convert("RGB")
    input_tensor = transform(image).unsqueeze(0).to(device)  # (1, 3, 224, 224)

    with torch.no_grad():
        outputs = model(input_tensor)
        _, predicted = torch.max(outputs, 1)
        predicted_class = class_names[predicted.item()]

        # 🔽 파일명 조합
        base_name, ext = os.path.splitext(os.path.basename(img_path))
        new_filename = f"{base_name}_{predicted_class}{ext}"
        save_path = os.path.join(output_dir, new_filename)

        # 🔽 이미지 저장
        image.save(save_path)

    print(f"📷 {os.path.basename(img_path)} → 예측 결과: {predicted_class}")


📷 cropped_20250702_135734_926_1343.jpg → 예측 결과: smoke
📷 cropped_20250702_135734_929_7663.jpg → 예측 결과: smoke
📷 cropped_20250702_135734_933_1296.jpg → 예측 결과: smoke
📷 cropped_20250702_135734_934_2336.jpg → 예측 결과: smoke
📷 cropped_20250702_135734_936_4357.jpg → 예측 결과: smoke
📷 cropped_20250702_135734_938_2019.jpg → 예측 결과: smoke
📷 cropped_20250702_135734_942_8939.jpg → 예측 결과: fire
📷 cropped_20250702_135734_946_1721.jpg → 예측 결과: smoke
📷 cropped_20250702_135734_948_7235.jpg → 예측 결과: smoke
📷 cropped_20250702_135734_950_5949.jpg → 예측 결과: smoke
📷 cropped_20250702_135734_956_5754.jpg → 예측 결과: smoke
📷 cropped_20250702_135734_958_6824.jpg → 예측 결과: smoke
📷 cropped_20250702_135734_959_3790.jpg → 예측 결과: smoke
📷 cropped_20250702_135734_961_3844.jpg → 예측 결과: smoke
📷 cropped_20250702_135734_963_2797.jpg → 예측 결과: smoke
📷 cropped_20250702_135734_965_9294.jpg → 예측 결과: smoke
📷 cropped_20250702_135734_966_8999.jpg → 예측 결과: smoke
📷 cropped_20250702_135734_968_3283.jpg → 예측 결과: smoke
📷 cropped_20250702_135734_970